# Setup Retail

To start we'll need to do some imports and load the [Jupysql](https://jupysql.ploomber.io/en/latest/quick-start.html) extension which allows is to conveniently write SQL directly in cells.

In [ ]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt

%load_ext sql

We'll also set some Jupysql configuration options. Other options can be viewed [here](https://jupysql.ploomber.io/en/latest/api/configuration.html).

In [ ]:
# Return the resultset as a pandas dataframe
%config SqlMagic.autopandas = True

# Verbosity level. 0=minimal, 1=normal, 2=all
%config SqlMagic.feedback = 0

# Show connection string after execution
%config SqlMagic.displaycon = False

%config SqlMagic.displaylimit = 10

Connnect to an **in-memory** duckdb database.

In [ ]:
%sql duckdb:///:default:

And now we are ready to do a little data wrangling.

The dataset we'll be using was downloaded from [Kaggle](https://www.kaggle.com/datasets/gabrielsantello/wholesale-and-retail-orders-dataset) and consists of a single csv containing order lines. Let's look at a sample.

In [ ]:
%sql select * from read_csv_auto('./data/retail_orders/orders.csv') limit 5;

It looks like some basic wrangling like renaming the columns, setting the date format and standardising the status column is needed. 

In [ ]:
%%sql

-- Read in the csv file and rename the columns and set the dateformat

create or replace table orders as select * from read_csv_auto(
    './data/retail_orders/orders.csv', 
    names=[
        'customer_id', 
        'status', 
        'order_date', 
        'delivery_date', 
        'order_id', 
        'product_id', 
        'qty', 
        'total', 
        'unit_cost', 
    ],
    skip=1,
    dateformat='%d-%b-%y'
);

-- Standardise the status column as lowercase

update orders set status = lower(status) where lower(status) <> status;